In [23]:
import pandas as pd
import re
from hazm import *

In [2]:
df = pd.read_csv('all_extracted_data.csv')
df.head()

,rowNumber,url,scrapedAt,trackingNumber,letterNumber,letterDate,newspaperNumber,newspaperDate,pageNumber,publishCount,title,content,companyName,companyNationalId,companyRegisterNumber,letterPublisher
0,1,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:19.485Z,020431317960184,140230417076000993,1402/04/31,23293 شهرستان,1403/12/21,69.0,1.0,آگهی تاسیس,تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...,پارس نيرو تابان انرژي,14012422050,12768,مرجع ثبت شرکت ها و موسسات غیرتجاری ماهشهر
1,2,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:19.719Z,030510715442872,140330400901062448,1403/05/10,23248 تهران,1403/10/26,57.0,1.0,آگهي ثبت تغيير شركت با مسئوليت محدود نگين كاسپ...,آگهي تغييرات شركت با مسئوليت محدود نگين كاسپين...,نگين كاسپين نفيس,14005328876,481539,اداره ثبت شرکت ها و موسسات غیرتجاری تهران
2,3,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:19.956Z,030730846200295,140330417078000449,1403/07/30,23245 شهرستان,1403/10/22,173.0,1.0,آگهي ثبت تغيير شركت سهامي خاص راه و ساختمان مه...,آگهي تغييرات شركت سهامي خاص راه و ساختمان مهر ...,راه و ساختمان مهر آ فرين گوهر افشان,10861781630,3410,مرجع ثبت شرکت ها و موسسات غیرتجاری شوشتر
3,4,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:20.223Z,030730745248760,140330417078000450,1403/07/30,23245 شهرستان,1403/10/22,130.0,1.0,آگهي ثبت تغيير شركت سهامي خاص راه و ساختمان مه...,آگهي تغييرات شركت سهامي خاص راه و ساختمان مهر ...,راه و ساختمان مهر آ فرين گوهر افشان,10861781630,3410,مرجع ثبت شرکت ها و موسسات غیرتجاری شوشتر
4,5,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:20.447Z,030730892899777,140330406185014365,1403/07/30,23242 شهرستان,1403/10/18,187.0,1.0,آگهي ثبت تغيير موسسه غير تجاري صندوق قرض الحسن...,آگهي تغييرات موسسه غير تجاري صندوق قرض الحسنه ...,صندوق قرض الحسنه رحمت واسعه رضاشهر,10380064201,443,اداره ثبت شرکت ها و موسسات غیرتجاری مشهد


In [3]:
# Check for duplicates in letterNumber column
duplicate_letters = df[df['letterNumber'].duplicated(keep=False)]
print(f"\nNumber of duplicate letterNumbers: {len(duplicate_letters)}")
print("\nDuplicate letterNumber entries:")
print(duplicate_letters[['letterNumber', 'companyName', 'letterDate']].sort_values('letterNumber'))



Number of duplicate letterNumbers: 3509

Duplicate letterNumber entries:
          letterNumber                   companyName  letterDate
10834    /ق301/3413/12                           NaN  1398/06/30
10833    /ق301/3413/12                           NaN  1398/06/30
3306   00137/41/87/ك‌ح                           NaN       -----
3305   00137/41/87/ك‌ح                           NaN  1389/03/29
3276           0033/95  گسترش فناوران آكام الكترونيك  1395/03/10
...                ...                           ...         ...
28827              NaN                           NaN       -----
29191              NaN                           NaN       -----
29205              NaN                           NaN       -----
30325              NaN                           NaN         NaN
30478              NaN                           NaN       -----

[3509 rows x 3 columns]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32011 entries, 0 to 32010
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   rowNumber              32011 non-null  int64  
 1   url                    32011 non-null  object 
 2   scrapedAt              32011 non-null  object 
 3   trackingNumber         31726 non-null  object 
 4   letterNumber           31891 non-null  object 
 5   letterDate             32000 non-null  object 
 6   newspaperNumber        31867 non-null  object 
 7   newspaperDate          31867 non-null  object 
 8   pageNumber             24237 non-null  float64
 9   publishCount           21114 non-null  float64
 10  title                  32000 non-null  object 
 11  content                32000 non-null  object 
 12  companyName            16920 non-null  object 
 13  companyNationalId      16920 non-null  object 
 14  companyRegisterNumber  16885 non-null  object 
 15  le

In [15]:
# Filter rows where title starts with 'آگهی تاسیس'
establishment_notices = df[df['title'].str.startswith('آگهی تاسیس', na=False)].reset_index(drop=True)
establishment_notices


,rowNumber,url,scrapedAt,trackingNumber,letterNumber,letterDate,newspaperNumber,newspaperDate,pageNumber,publishCount,title,content,companyName,companyNationalId,companyRegisterNumber,letterPublisher
0,1,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:19.485Z,020431317960184,140230417076000993,1402/04/31,23293 شهرستان,1403/12/21,69.0,1.0,آگهی تاسیس,تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...,پارس نيرو تابان انرژي,14012422050,12768,مرجع ثبت شرکت ها و موسسات غیرتجاری ماهشهر
1,8,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:21.139Z,030822671644939,140330418571004212,1403/08/22,23208 شهرستان,1403/09/07,102.0,1.0,آگهی تاسیس,تاسيس شركت با مسئوليت محدود خدمات پيشرو گيل گس...,خدمات پيشرو گيل گستر مهيار,14013938535,25761,اداره ثبت شرکت ها و موسسات غیرتجاری رشت
2,10,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:21.890Z,030728684571476,140330400901097372,1403/07/28,23206 تهران,1403/09/05,50.0,1.0,آگهی تاسیس,تاسيس شركت سهامي خاص تهيه و توزيع ابزار تجارت ...,تهيه و توزيع ابزار تجارت صانع,14013840308,639821,اداره ثبت شرکت ها و موسسات غیرتجاری تهران
3,12,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:22.488Z,030806630103731,140330400901102009,1403/08/06,23206 تهران,1403/09/05,29.0,1.0,آگهی تاسیس,تاسيس شركت با مسئوليت محدود توسعه تجارت دايان ...,توسعه تجارت دايان پارس,14013870808,640376,اداره ثبت شرکت ها و موسسات غیرتجاری تهران
4,14,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-30T08:34:23.620Z,030823653351875,140330418571004268,1403/08/22,23205 شهرستان,1403/09/04,45.0,1.0,آگهی تاسیس,تاسيس شركت سهامي خاص نوشت افزار رزاق درتاريخ 2...,نوشت افزار رزاق,14013936571,25759,اداره ثبت شرکت ها و موسسات غیرتجاری رشت
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4888,274,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-29T08:05:55.066Z,000512156837323,140030406185010088,1400/05/11,22292 شهرستان,1400/07/07,8.0,1.0,آگهی تاسیس,تاسيس شركت با مسئوليت محدود آيرين شهد اميد درت...,آيرين شهد اميد,14010193199,77750,اداره ثبت شرکت ها و موسسات غیرتجاری مشهد
4889,337,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-29T08:06:15.605Z,000429240763139,140030419588000660,1400/04/29,22272 شهرستان,1400/06/14,11.0,1.0,آگهی تاسیس,تاسيس شركت سهامي خاص مهر ميترا سمنگان درتاريخ ...,مهر ميترا سمنگان,14010174820,5292,مرجع ثبت شرکت ها و موسسات غیرتجاری سیرجان
4890,343,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-29T08:06:17.462Z,000608246947830,140030411137006894,1400/06/08,22270 شهرستان,1400/06/11,26.0,1.0,آگهی تاسیس,تاسيس شركت تعاوني كارآفرينان ميهن دوست نيايش د...,كارآفرينان ميهن دوست نيايش,14010234720,55934,اداره ثبت شرکت ها و موسسات غیرتجاری شیراز
4891,416,https://rrk.ir/ords/r/rrs/rrs-front/f-detail-a...,2025-04-29T08:06:43.027Z,000516813631549,140030427086000218,1400/05/16,22251 شهرستان,1400/05/18,36.0,1.0,آگهی تاسیس,تاسيس شركت با مسئوليت محدود خانه اميد خدابنده ...,خانه اميد خدابنده,14010198214,1871,مرجع ثبت شرکت ها و موسسات غیرتجاری خدابنده


In [8]:
# Get unique values in title column and display them
unique_titles = df['title'].unique()
len(unique_titles)
unique_titles


array(['آگهی تاسیس',
       'آگهي ثبت تغيير شركت با مسئوليت محدود نگين كاسپين نفيس به شماره ثبت 481539 و شناسه ملی 14005328876',
       'آگهي ثبت تغيير شركت سهامي خاص راه و ساختمان مهر آ فرين گوهر افشان به شماره ثبت 3410 و شناسه ملی 10861781630',
       ...,
       'آگهی تغییرات شرکت مانا رایان کاوان شرکت سهامی خاص به شماره ثبت 2693 و شناسه ملی 14008764891',
       'آگهی تغییرات شرکت مولدسازی و مدیریت دارایی تجارت شرکت سهامی خاص به شماره ثبت 447895 و شناسه ملی 14003820977',
       'آگهی تغییرات شرکت گل آرد زاگرس شرکت سهامی خاص به شماره ثبت 150 و شناسه ملی 14003663523'],
      dtype=object)

In [18]:

text =establishment_notices['content'][1]

# Normalize
normalizer = Normalizer()

def extract_people(text):
    text = normalizer.normalize(text)
    pattern = r"آقای\s([\w\s]+)\sبه شماره ملی\s(\d+)\sبه سمت\s([\w\s]+)"
    matches = re.findall(pattern, text)

    people = []
    for name, national_id, role in matches:
        people.append({
            'name': name.strip(),
            'national_id': national_id,
            'role': role.strip()
        })
    return people


extract_people(text)

[{'name': 'محمدابراهیم مهیار',
  'national_id': '۲۵۹۰۳۴۵۰۳۸',
  'role': 'رئیس هیئت'}]

In [ ]:
# Apply the extraction to all rows and build the new dataframe
extracted_rows = []

for idx, row in establishment_notices.iterrows():
    text = row['content']
    people = extract_people(text)
    for person in people:
        extracted_rows.append({
            'note_id': idx,
            'content': text,
            'name': person['name'],
            'national_id': person['national_id'],
            'role': person['role']
        })

# Create new DataFrame
people_df = pd.DataFrame(extracted_rows)

# View the results
people_df.head()

   note_id                                            content  \
0        0  تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...   
1        0  تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...   
2        0  تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...   
3        0  تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...   
4        0  تاسیس شرکت سهامی خاص پارس نیرو تابان انرژی درت...   

                  name national_id  \
0  تورج طاهری عبده وند  ۱۹۷۱۵۴۷۱۳۱   
1  مهدی ظاهری عبده وند  ۱۹۷۲۳۱۰۵۸۵   
2   پیام دستگیرزوردگان  ۱۹۷۲۳۳۱۳۵۳   
3       علیمحمد بیگدلی  ۱۹۰۰۰۹۹۲۷۶   
4          امید الیاسی  ۱۹۷۲۳۰۹۸۵۴   

                                                role  
0                                      عضو اصلی هیئت  
1                                          رئیس هیئت  
2                                               نایب  
3                                          بازرس علی  
4  بازرس اصلی به مدت ۱ سال روزنامه کثیر الانتشار ...  


In [27]:
people_df[['note_id', 'name','national_id','role']].to_csv('./people.csv')

In [22]:
def extract_company_info(text):
    company_pattern = r"تاسیس شرکت (.*?) درتاریخ"
    company_match = re.search(company_pattern, text)
    company_name = company_match.group(1).strip() if company_match else None

    reg_no = re.search(r"شماره ثبت\s(\d+)", text)
    national_id = re.search(r"شناسه ملی\s(\d+)", text)

    company_info = {
        'company_name': company_name,
        'registration_number': reg_no.group(1) if reg_no else None,
        'national_id': national_id.group(1) if national_id else None,
    }

    return company_info

extract_company_info(text)

{'company_name': None, 'registration_number': '13910', 'national_id': None}

In [40]:
dfc = pd.read_csv('all_company_data.csv')
dfc.head()

/var/folders/3l/rrp_284j66z_p7ybmxxh30yr0000gn/T/ipykernel_63343/1153245638.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfc = pd.read_csv('all_company_data.csv')


,CompanyId,CompanyName,NationalId,RegistrationNumber,PostalCode,Address
0,327629,آلما صنعت سپنتا,10320752871,422383,NaN,NaN
1,1637305,آرون تدبير ساويس,14009860205,19390,NaN,NaN
2,742786,مخابرات آسمان,10101510352,107174,NaN,NaN
3,1565487,آركا ماشين رايمون,14009274598,3689,NaN,NaN
4,1586161,آروين صنعت اسرار,14009439582,4518,NaN,NaN


In [29]:
dfc

,CompanyId,CompanyName,NationalId,RegistrationNumber,PostalCode,Address
0,327629,آلما صنعت سپنتا,10320752871,422383,NaN,NaN
1,1637305,آرون تدبير ساويس,14009860205,19390,NaN,NaN
2,742786,مخابرات آسمان,10101510352,107174,NaN,NaN
3,1565487,آركا ماشين رايمون,14009274598,3689,NaN,NaN
4,1586161,آروين صنعت اسرار,14009439582,4518,NaN,NaN
...,...,...,...,...,...,...
10410704,2000351,هیراک کیش,10861546522,3016,NaN,NaN
10410705,1938582,فیدار آرتین آذرخش اروند,14012857113,9656,NaN,NaN
10410706,536920,آگهی تغییرات شرکت بازرگانی جام جم منطقه آزاد ...,10980187788,1091,NaN,NaN
10410707,441024,ائل باخچاسی ارس,14003959217,20,NaN,NaN


In [43]:
dfc = dfc.drop_duplicates(subset=['RegistrationNumber']).reset_index(drop=True)

In [44]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381521 entries, 0 to 381520
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   CompanyId           381521 non-null  object
 1   CompanyName         381521 non-null  object
 2   NationalId          380973 non-null  object
 3   RegistrationNumber  381520 non-null  object
 4   PostalCode          85948 non-null   object
 5   Address             85953 non-null   object
dtypes: object(6)
memory usage: 17.5+ MB


In [45]:
dfc.to_csv('./companies.csv')

In [47]:
dfc[['CompanyName', 'RegistrationNumber', 'NationalId', 'CompanyId', 'PostalCode', 'Address']].to_csv('./companies_2.csv', index=False)